# Quantum Convolutional Neural Network

This is an implementation of 8 Qubit QCNN circuit (Quantum Convolutional Neural Networks by Iris Cong et al, 2019) and Tree Tensor Network circuit (Hierarchical Quantum Classifier by Ed Grant et al, 2018) with the comparisons among various ansatze.

In [1]:
import pennylane as qml
from pennylane import numpy as np
import statistics as st
import random
import os
np.random.seed(0)

import sklearn.datasets
import tensorflow as tf
#import pandas as pd

/home/leeseok/.local/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# QCNN Circuit

Here are various unitary ansatze to test

In [2]:
# Unitraies for Convolutional Layers 
def U_TTN(params, wires): # 2 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

def U_5(params, wires): # 10 params
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])
    qml.RZ(params[2], wires = wires[0])
    qml.RZ(params[3], wires = wires[1])
    qml.CRZ(params[4], wires = [wires[1], wires[0]])
    qml.CRZ(params[5], wires = [wires[0], wires[1]])
    qml.RX(params[6], wires = wires[0])
    qml.RX(params[7], wires = wires[1])
    qml.RZ(params[8], wires = wires[0])
    qml.RZ(params[9], wires = wires[1])
    
def U_6(params, wires): # 10 params
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])
    qml.RZ(params[2], wires = wires[0])
    qml.RZ(params[3], wires = wires[1])
    qml.CRX(params[4], wires = [wires[1], wires[0]])
    qml.CRX(params[5], wires = [wires[0], wires[1]])
    qml.RX(params[6], wires = wires[0])
    qml.RX(params[7], wires = wires[1])
    qml.RZ(params[8], wires = wires[0])
    qml.RZ(params[9], wires = wires[1])

def U_9(params, wires): # 2 params
    qml.Hadamard(wires = wires[0])
    qml.Hadamard(wires = wires[1])
    qml.CZ(wires = [wires[0],wires[1]])
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])

def U_13(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CRZ(params[2], wires = [wires[1], wires[0]])
    qml.RY(params[3], wires = wires[0])
    qml.RY(params[4], wires = wires[1])
    qml.CRZ(params[5], wires = [wires[0], wires[1]])

def U_14(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CRX(params[2], wires = [wires[1], wires[0]])
    qml.RY(params[3], wires = wires[0])
    qml.RY(params[4], wires = wires[1])
    qml.CRX(params[5], wires = [wires[0], wires[1]])

def U_15(params, wires): # 4 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[1], wires[0]])
    qml.RY(params[2], wires = wires[0])
    qml.RY(params[3], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

def U_SO4(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])
    qml.RY(params[2], wires = wires[0])
    qml.RY(params[3], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])
    qml.RY(params[4], wires = wires[0])
    qml.RY(params[5], wires = wires[1])

# Unitraies for Pooling and Fully Connected Layers
def V_0(theta, wires):
    qml.CRZ(theta, wires = [wires[0], wires[1]])

def V_1(theta, wires):
    qml.CRX(theta, wires = [wires[0], wires[1]])

def F(theta, wires):
    qml.CRZ(theta, wires = [wires[0], wires[1]])

Below is general circuit structures used in Quantum Convolutional Neural Network (QCNN) circuits.

In [3]:
# Convolution Layer1
def conv_layer1(U, params):
    U(params, wires = [0,7])
    for i in range (0,8,2):
        U(params, wires = [i, i+1])
    for i in range (1,7,2):
        U(params, wires = [i, i+1])
    
def conv_layer2(U, params):
    U(params, wires = [0,2])
    U(params, wires = [4,6])
    U(params, wires = [2,4])
    U(params, wires = [0,6])
    
def pooling_layer1(V_0, V_1, params):
    for i in range(0,8,2):
        V_0(params[0], wires = [i+1, i])
    for i in range(0,8,2):
        qml.PauliX(wires = i+1)
    for i in range(0,8,2):
        V_1(params[1], wires = [i+1, i])
        

def pooling_layer2(V_0, V_1, params): # 2params
    V_0(params[0], wires = [2,0])
    V_0(params[0], wires = [6,4])
    
    qml.PauliX(wires = 2)
    qml.PauliX(wires = 6)
    
    V_1(params[1], wires = [2,0])
    V_1(params[1], wires = [6,4])
    
def pooling_layer3(V_0, V_1, params):
    V_0(params[0], wires = [0,4])
    qml.PauliX(wires = 0)
    V_1(params[1], wires = [0,4])

Here we define various possible embedding methods.

In [4]:
from pennylane.templates.embeddings import AmplitudeEmbedding, AngleEmbedding

def data_embedding(X, embedding_type = 'Amplitude'):
    if embedding_type == 'Amplitude':
        AmplitudeEmbedding(X, wires = range(8), normalize = True)
    elif embedding_type == 'Angle':
        AngleEmbedding(X, wires = range(8), rotation = 'Y')
    elif embedding_type == "Compact":
        AngleEmbedding(X[:8], wires = range(8), rotation = 'X')
        AngleEmbedding(X[8:], wires = range(8), rotation = 'Y')
        #AngleEmbedding(X[16:], wires = range(8), rotation = 'Z')

Define QCNN circuit with given Unitary Ansatz and embedding method.

In [5]:
dev = qml.device('default.qubit', wires = 8)

@qml.qnode(dev)
def QCNN(X, params, U, U_params, embedding_type = 'Amplitude'):

    param1 = params[0:U_params]
    param2 = params[U_params:U_params + 2]
    param3 = params[U_params + 2: 2*U_params + 2]
    param4 = params[2*U_params + 2: 2*U_params + 4]
    param5 = params[2*U_params + 4: 3*U_params + 4]
    param6 = params[3*U_params+4:]
    
    # Data Embedding
    data_embedding(X, embedding_type = embedding_type)
    
    #Quantum Convolutional Neural Network
    if U == 'U_TTN':
        conv_layer1(U_TTN, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_TTN, param3)
        pooling_layer2(V_0, V_1, param4)
        U_TTN(param5, wires = [0,4])
        pooling_layer3(V_0, V_1, param6)
        
    elif U == 'U_5':
        conv_layer1(U_5, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_5, param3)
        pooling_layer2(V_0, V_1, param4)
        U_5(param5, wires = [0,4])
        pooling_layer3(V_0, V_1, param6)
        
    elif U == 'U_6':
        conv_layer1(U_6, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_6, param3)
        pooling_layer2(V_0, V_1, param4)
        U_6(param5, wires = [0,4])
        pooling_layer3(V_0, V_1, param6)
        
    elif U == 'U_9':
        conv_layer1(U_9, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_9, param3)
        pooling_layer2(V_0, V_1, param4)
        U_9(param5, wires = [0,4])
        pooling_layer3(V_0, V_1, param6)
        
    elif U == 'U_13':
        conv_layer1(U_13, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_13, param3)
        pooling_layer2(V_0, V_1, param4)
        U_13(param5, wires = [0,4])
        pooling_layer3(V_0, V_1, param6)
        
    elif U == 'U_14':
        conv_layer1(U_14, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_14, param3)
        pooling_layer2(V_0, V_1, param4)
        U_14(param5, wires = [0,4])
        pooling_layer3(V_0, V_1, param6)
        
    elif U == 'U_15':
        conv_layer1(U_15, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_15, param3)
        pooling_layer2(V_0, V_1, param4)
        U_15(param5, wires = [0,4])
        pooling_layer3(V_0, V_1, param6) 
        
    elif U == 'U_SO4':
        conv_layer1(U_SO4, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_SO4, param3)
        pooling_layer2(V_0, V_1, param4)
        U_SO4(param5, wires = [0,4])
        pooling_layer3(V_0, V_1, param6)    
        
    return qml.expval(qml.PauliZ(4))

Below is Hierarchical Quantum Classifier structure with different Ansatze

In [6]:
dev_TTN = qml.device('default.qubit', wires = 8)

@qml.qnode(dev_TTN)
def Hierarchical_classifier(X, params, U, U_params, embedding_type = 'Amplitude'):
    
    param1 = params[0 * U_params:1 * U_params]
    param2 = params[1 * U_params:2 * U_params]
    param3 = params[2 * U_params:3 * U_params]
    param4 = params[3 * U_params:4 * U_params]
    param5 = params[4 * U_params:5 * U_params]
    param6 = params[5 * U_params:6 * U_params]
    param7 = params[6 * U_params:7 * U_params]
    
    data_embedding(X, embedding_type = embedding_type)
    ['U_TTN', 'U_5', 'U_6', 'U_13', 'U_14', 'U_15', 'U_SO4'] 
    if U == 'U_TTN':
        # layer 1
        U_TTN(param1, wires = [0,1])
        U_TTN(param2, wires = [2,3])
        U_TTN(param3, wires = [4,5])
        U_TTN(param4, wires = [6,7])
        # layer 2
        U_TTN(param5, wires = [1,3])
        U_TTN(param6, wires = [5,7])
        # layer 3
        U_TTN(param7, wires = [3,7])
    elif U == 'U_5':
        # layer 1
        U_5(param1, wires = [0,1])
        U_5(param2, wires = [2,3])
        U_5(param3, wires = [4,5])
        U_5(param4, wires = [6,7])
        # layer 2
        U_5(param5, wires = [1,3])
        U_5(param6, wires = [5,7])
        # layer 3
        U_5(param7, wires = [3,7])
    elif U == 'U_6':
        # layer 1
        U_6(param1, wires = [0,1])
        U_6(param2, wires = [2,3])
        U_6(param3, wires = [4,5])
        U_6(param4, wires = [6,7])
        # layer 2
        U_6(param5, wires = [1,3])
        U_6(param6, wires = [5,7])
        # layer 3
        U_6(param7, wires = [3,7])
    elif U == 'U_13':
        # layer 1
        U_13(param1, wires = [0,1])
        U_13(param2, wires = [2,3])
        U_13(param3, wires = [4,5])
        U_13(param4, wires = [6,7])
        # layer 2
        U_13(param5, wires = [1,3])
        U_13(param6, wires = [5,7])
        # layer 3
        U_13(param7, wires = [3,7])
    elif U == 'U_14':
        # layer 1
        U_14(param1, wires = [0,1])
        U_14(param2, wires = [2,3])
        U_14(param3, wires = [4,5])
        U_14(param4, wires = [6,7])
        # layer 2
        U_14(param5, wires = [1,3])
        U_14(param6, wires = [5,7])
        # layer 3
        U_14(param7, wires = [3,7])
    elif U == 'U_15':
        # layer 1
        U_15(param1, wires = [0,1])
        U_15(param2, wires = [2,3])
        U_15(param3, wires = [4,5])
        U_15(param4, wires = [6,7])
        # layer 2
        U_15(param5, wires = [1,3])
        U_15(param6, wires = [5,7])
        # layer 3
        U_15(param7, wires = [3,7])
    elif U == 'U_SO4':
        # layer 1
        U_SO4(param1, wires = [0,1])
        U_SO4(param2, wires = [2,3])
        U_SO4(param3, wires = [4,5])
        U_SO4(param4, wires = [6,7])
        # layer 2
        U_SO4(param5, wires = [1,3])
        U_SO4(param6, wires = [5,7])
        # layer 3
        U_SO4(param7, wires = [3,7])
    

    return qml.expval(qml.PauliZ(7))

# Training Quantum Circuits

In [7]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def cost(params, X, Y, U, U_params, embedding_type, circuit):
    if circuit == 'QCNN':
        predictions = [QCNN(x, params, U, U_params, embedding_type) for x in X]
    elif circuit == 'Hierarchical':
        predictions = [Hierarchical_classifier(x, params, U, U_params, embedding_type) for x in X]
    
    return square_loss(Y, predictions)

def accuracy_test_binary(predictions, labels):
    acc = 0
    for l,p in zip(labels, predictions):
        if np.abs(l - p) < 1:
            acc = acc + 1
    return acc / len(labels)

def accuracy_test_one_class(predictions, labels):
    acc = 0
    for l,p in zip(labels, predictions):
        if np.abs(l - p) < 0.5:
            acc = acc + 1
    return acc / len(labels)

In [8]:
def circuit_training(X_train, Y_train, X_val, Y_val, U, U_params, embedding_type, circuit):
    if circuit == 'QCNN':
        total_params = U_params * 3 + 2 * 3
    elif circuit == 'Hierarchical':
        total_params = U_params * 7
        
    params = np.random.randn(total_params)
    steps = 400
    learning_rate = 0.01
    batch_size = 25
    opt = qml.NesterovMomentumOptimizer(learning_rate)
    validation=[]
    
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, U, U_params, embedding_type, circuit), params)
        
        if it % 10 == 0:
            predictions = [QCNN(x, params, U, U_params, embedding_type) for x in X_val]
            validation.append(accuracy_test_binary(predictions, Y_val))
            print(validation)
            
            if len(validation) > 3:
                if (validation[it//10] < validation[it//10 - 3]) & (validation[it//10] < validation[it//10 - 2]) & (validation[it//10] < validation[it//10 - 1]):
                    break
                    
        print("iteration: ", it, " cost: ", cost_new)
        
        
    return params

### MNIST Data loading and processing

Use PCA and Autoencoder to reduce it into 8 features. We test both one-class classification (labeling 0 and 1) and binary classification (labeling -1 and 1).

In [9]:
import tensorflow as tf
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses

def data_load_and_process(classes = [0,1], feature_reduction = 'resize256', binary = True):
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0 #normalize the data
    
    x_train_filter_01 = np.where((y_train == classes[0]) | (y_train == classes[1]))
    x_test_filter_01 = np.where((y_test == classes[0]) | (y_test == classes[1]))
    
    x_train_01, x_test_01 = x_train[x_train_filter_01], x_test[x_test_filter_01]
    y_train_01, y_test_01 = y_train[x_train_filter_01], y_test[x_test_filter_01]
    
    if binary == False:
        y_train_01 = [1 if y ==classes[0] else 0 for y in y_train_01]
        y_test_01 = [1 if y ==classes[0] else 0 for y in y_test_01]
    elif binary == True:
        y_train_01 = [1 if y ==classes[0] else -1 for y in y_train_01]
        y_test_01 = [1 if y ==classes[0] else -1 for y in y_test_01]
        
    
    if feature_reduction == 'resize256':   
        x_train_01 = tf.image.resize(x_train_01[:], (256, 1)).numpy()
        x_test_01 = tf.image.resize(x_test_01[:], (256, 1)).numpy()
        x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01) 
        return x_train_01, x_test_01, y_train_01, y_test_01
    
    elif feature_reduction == 'pca8':
        x_train_01 = tf.image.resize(x_train_01[:], (784, 1)).numpy()
        x_test_01 = tf.image.resize(x_test_01[:], (784, 1)).numpy()
        x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01)
        
        pca = PCA(8)
        x_train_01 = pca.fit_transform(x_train_01)
        x_test_01 = pca.transform(x_test_01)
        
        #Rescale for angle embedding
        x_train_01, x_test_01 = (x_train_01 + 10) * (np.pi / 20), (x_test_01 + 10) * (np.pi / 20)
        
        return x_train_01, x_test_01, y_train_01, y_test_01
    
    elif feature_reduction == 'pca16':
        x_train_01 = tf.image.resize(x_train_01[:], (784, 1)).numpy()
        x_test_01 = tf.image.resize(x_test_01[:], (784, 1)).numpy()
        x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01)
        
        pca = PCA(16)
        x_train_01 = pca.fit_transform(x_train_01)
        x_test_01 = pca.transform(x_test_01)
        
        #Rescale for angle embedding
        x_train_01, x_test_01 = (x_train_01 + 10) * (np.pi / 20), (x_test_01 + 10) * (np.pi / 20)
        
        return x_train_01, x_test_01, y_train_01, y_test_01

    elif feature_reduction == 'autoencoder8':
        latent_dim = 8 
        class Autoencoder(Model):
            def __init__(self, latent_dim):
                super(Autoencoder, self).__init__()
                self.latent_dim = latent_dim   
                self.encoder = tf.keras.Sequential([
                layers.Flatten(),
                  layers.Dense(latent_dim, activation='relu'),
                ])
                self.decoder = tf.keras.Sequential([
                layers.Dense(784, activation='sigmoid'),
                layers.Reshape((28, 28))
                ])
            def call(self, x):
                encoded = self.encoder(x)
                decoded = self.decoder(encoded)
                return decoded
        
        autoencoder = Autoencoder(latent_dim)
        
        autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
        autoencoder.fit(x_train_01, x_train_01,
                epochs=10,
                shuffle=True,
                validation_data=(x_test_01, x_test_01))
        
        x_train_01, x_test_01 = autoencoder.encoder(x_train_01).numpy(), autoencoder.encoder(x_test_01).numpy()
        #Rescale for Angle Embedding
        x_train_01, x_test_01 = x_train_01 * (np.pi / 50), x_test_01 * (np.pi / 50)
        
        return x_train_01, x_test_01, y_train_01, y_test_01
    
    elif feature_reduction == 'autoencoder16':
        latent_dim = 16 
        class Autoencoder(Model):
            def __init__(self, latent_dim):
                super(Autoencoder, self).__init__()
                self.latent_dim = latent_dim   
                self.encoder = tf.keras.Sequential([
                layers.Flatten(),
                  layers.Dense(latent_dim, activation='relu'),
                ])
                self.decoder = tf.keras.Sequential([
                layers.Dense(784, activation='sigmoid'),
                layers.Reshape((28, 28))
                ])
            def call(self, x):
                encoded = self.encoder(x)
                decoded = self.decoder(encoded)
                return decoded
        
        autoencoder = Autoencoder(latent_dim)
        
        autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
        autoencoder.fit(x_train_01, x_train_01,
                epochs=10,
                shuffle=True,
                validation_data=(x_test_01, x_test_01))
        
        x_train_01, x_test_01 = autoencoder.encoder(x_train_01).numpy(), autoencoder.encoder(x_test_01).numpy()
        #Rescale for Angle Embedding
        x_train_01, x_test_01 = x_train_01 * (np.pi / 50), x_test_01 * (np.pi / 50)
        
        return x_train_01, x_test_01, y_train_01, y_test_01

# Results

Benchmarking function trains and performs accuracy tests for all the given unitary ansatze, feature reduction methods, and circuit structures.

In [10]:
def Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = True):
    I = len(Unitaries)
    J = len(Encodings)
    All_predictions = []

    accuracy_list = []
    
    for step in range(5):
        print(str(step) + "th step : ")
        for i in range(I):
            for j in range(J):
                U = Unitaries[i]
                U_params = U_num_params[i]
                Encoding = Encodings[j]
                if Encoding == 'resize256':
                    Embedding = 'Amplitude'
                    X_train, X_test, Y_train, Y_test = data_load_and_process(classes = classes, feature_reduction = 'resize256', binary = binary)
                elif Encoding == 'pca8':
                    Embedding = 'Angle'
                    X_train, X_test, Y_train, Y_test = data_load_and_process(classes = classes, feature_reduction = 'pca8', binary = binary)
                elif Encoding == 'pca16':
                    Embedding = 'Compact'
                    X_train, X_test, Y_train, Y_test = data_load_and_process(classes = classes, feature_reduction = 'pca16', binary = binary)
                elif Encoding == 'autoencoder8':
                    Embedding = 'Angle'
                    X_train, X_test, Y_train, Y_test = data_load_and_process(classes = classes, feature_reduction = 'autoencoder8', binary = binary)
                elif Encoding == 'autoencoder16':
                    Embedding = 'Compact'
                    X_train, X_test, Y_train, Y_test = data_load_and_process(classes = classes, feature_reduction = 'autoencoder16', binary = binary)

                    
                #Make validation data by randomly extracting 1000 data from X_train, Y_train
                randomlist = random.sample(range(0, len(X_train)), 500)
                X_val = [X_train[r] for r in randomlist]
                Y_val = [Y_train[r] for r in randomlist]
                
                X_train = [i for j, i in enumerate(X_train) if j not in randomlist]
                Y_train = [i for j, i in enumerate(Y_train) if j not in randomlist]
                
                print("\n")
                print("Loss History for " + circuit + " circuits, "+ U + " " + Encoding)
                trained_params = circuit_training(X_train, Y_train, X_val, Y_val, U, U_params, Embedding, circuit)

                if circuit == 'QCNN':
                    predictions = [QCNN(x, trained_params, U, U_params, Embedding) for x in X_test]
                elif circuit == 'Hierarchical':
                    predictions = [Hierarchical_classifier(x, trained_params, U, U_params, Embedding) for x in X_test]


                if binary == True:
                    accuracy = accuracy_test_binary(predictions, Y_test)
                elif binary == False:
                    accuracy = accuracy_test_one_class(predictions, Y_test)
                
                accuracy_list.append(accuracy) #store results

                print("Accuracy for " + U + " " + Encoding + " :" + str(accuracy))
                
    print(accuracy_list)

### 1. Results for QCNN circuit

#### 1.1 Binary Classification with 1, -1 labels

In [11]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_9', 'U_13', 'U_14', 'U_15', 'U_SO4'] 
U_num_params = [2, 10, 10, 2, 6, 6, 4, 6]
Encodings = ['resize256', 'pca8', 'pca16', 'autoencoder8', 'autoencoder16']
classes = [0,1]
circuit = 'QCNN'

In [ ]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = True)

0th step : 


Loss History for QCNN circuits, U_TTN resize256
[0.544]
iteration:  0  cost:  0.9268568666490181
iteration:  1  cost:  0.9165087067919642
iteration:  2  cost:  0.9494373721693371
iteration:  3  cost:  0.7562196642191972
iteration:  4  cost:  1.025579576773208
iteration:  5  cost:  0.9116733315899882
iteration:  6  cost:  0.8179514982703104
iteration:  7  cost:  0.8544212710272064
iteration:  8  cost:  0.8623752109877917
iteration:  9  cost:  0.8783832183364231
[0.544, 0.55]
iteration:  10  cost:  0.827265819036497
iteration:  11  cost:  0.7727733129974065
iteration:  12  cost:  0.9058695684596588
iteration:  13  cost:  0.8218012022050912
iteration:  14  cost:  0.9286926551412488
iteration:  15  cost:  0.7983076739185678
iteration:  16  cost:  0.8108123033484909
iteration:  17  cost:  0.9143155894221171
iteration:  18  cost:  0.9381777074037468
iteration:  19  cost:  0.9093346618185997
[0.544, 0.55, 0.552]
iteration:  20  cost:  0.9226175449813039
iteration:  21  cost:  0.

In [34]:
results = [0.9163120567375886, 0.8912529550827423, 0.9853427895981087, 0.9167848699763593, 0.9806146572104019, 0.9858156028368794, 0.9858156028368794, 0.8695035460992908, 0.9843971631205674, 0.9238770685579196, 0.975886524822695, 0.9163120567375886, 0.9702127659574468, 0.9290780141843972, 0.9867612293144208, 0.9442080378250591, 0.900709219858156, 0.7598108747044917, 0.9886524822695035, 0.9040189125295508, 0.9829787234042553, 0.908274231678487, 0.5617021276595745, 0.8921985815602836, 0.9858156028368794, 0.9210401891252955, 0.9706855791962175, 0.9219858156028369, 0.9801418439716312, 0.8969267139479905, 0.9810874704491725, 0.9626477541371158, 0.8704491725768322, 0.9588652482269504, 0.9891252955082742, 0.8713947990543736, 0.9853427895981087, 0.9536643026004729, 0.9224586288416076, 0.9286052009456265, 0.9853427895981087, 0.9640661938534278, 0.9858156028368794, 0.9366430260047282, 0.9829787234042553, 0.9442080378250591, 0.9843971631205674, 0.8832151300236407, 0.9517730496453901, 0.9215130023640662, 0.9560283687943263, 0.9437352245862884, 0.9872340425531915, 0.9484633569739953, 0.9735224586288416, 0.8458628841607565, 0.9796690307328605, 0.9541371158392435, 0.9825059101654846, 0.8940898345153664, 0.9820330969267139, 0.9361702127659575, 0.9782505910165484, 0.9475177304964539, 0.9309692671394799, 0.915839243498818, 0.9820330969267139, 0.9427895981087471, 0.9787234042553191, 0.9687943262411347, 0.7990543735224587, 0.7550827423167848, 0.9820330969267139, 0.9352245862884161, 0.9773049645390071, 0.9560283687943263, 0.9825059101654846, 0.9139479905437352, 0.9853427895981087, 0.9304964539007092]

In [35]:
len(results)

80

In [36]:
results_list = []
for i in range(16):
    #print(results[i], results[i+21], results[i+2*21], results[i+3*21], results[i+4*21])
    results_list.append([results[i], results[i+16], results[i+2*16], results[i+3*16], results[i+4*16]])

In [37]:
import statistics as st

In [38]:
print("Results for QNN with Compact Embedding (Rx-Ry) with 5 random initializations")
print("")
for i in range(16):
    if i < 2:
        it = "U_TTN"
    elif 1<i<4:
        it = 'U_5'
    elif 3<i<6:
        it = 'U_6'
    elif 5<i<8:
        it = 'U_9'
    elif 7<i<10:
        it = 'U_13'
    elif 9<i<12:
        it = 'U_14'
    elif 11<i<14:
        it = 'U_15'
    else:
        it = 'SO(4)'
    
    if i % 2 == 0:
        emb = "pca16"
    else:
        emb = "Autoencoding16"
        
    print(it + " " + emb + " Mean : " + str(st.mean(results_list[i])),  it + " " + emb + " Variance : " + str(st.variance(results_list[i])))

Results for QNN with Compact Embedding (Rx-Ry) with 5 random initializations

U_TTN pca16 Mean : 0.9140425531914894 U_TTN pca16 Variance : 0.0009483538163182047
U_TTN Autoencoding16 Mean : 0.8894562647754137 U_TTN Autoencoding16 Variance : 0.005838114559406247
U_5 pca16 Mean : 0.9802364066193854 U_5 pca16 Variance : 0.00019129375338821465
U_5 Autoencoding16 Mean : 0.9157446808510639 U_5 Autoencoding16 Variance : 0.0009050964348987589
U_6 pca16 Mean : 0.9829787234042553 U_6 pca16 Variance : 1.1848275014111777e-05
U_6 Autoencoding16 Mean : 0.9530023640661939 U_6 Autoencoding16 Variance : 0.0008369353207138023
U_9 pca16 Mean : 0.8485106382978723 U_9 pca16 Variance : 0.031163489428767826
U_9 Autoencoding16 Mean : 0.8582505910165484 U_9 Autoencoding16 Variance : 0.004256481621201707
U_13 pca16 Mean : 0.983451536643026 U_13 pca16 Variance : 6.594794583326366e-06
U_13 Autoencoding16 Mean : 0.9396690307328606 U_13 Autoencoding16 Variance : 0.0003551799875928438
U_14 pca16 Mean : 0.978439716312

#### 1.2 One Class Classification with labels 0,1

In [ ]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_9', 'U_13', 'U_14', 'U_15', 'U_SO4']
U_num_params = [2, 10, 10, 4, 6, 6, 4, 6]
Encodings = ['resize256', 'pca8', 'autoencoder8']
classes = [0,1]
circuit = 'QCNN'

In [ ]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = False)

#### 1.3 Binary Classification with 1, -1 labels with only Compact Encoding

In [132]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_9', 'U_13', 'U_14', 'U_15', 'U_SO4']
U_num_params = [2, 10, 10, 4, 6, 6, 4, 6]
Encodings = ['pca16', 'autoencoder16']
classes = [0,1]
circuit = 'QCNN'

In [ ]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = True)

0th step : 


Loss History for QCNN circuits, U_TTN pca16
iteration:  0  cost:  1.02462241418309
iteration:  10  cost:  0.939968561705772
iteration:  20  cost:  0.8299348593435605
iteration:  30  cost:  0.7669434726408423
iteration:  40  cost:  0.8238425968585373
iteration:  50  cost:  0.7961637973948912
iteration:  60  cost:  0.8097759590534136
iteration:  70  cost:  0.8776352478438227
iteration:  80  cost:  0.8070919063647426
iteration:  90  cost:  0.7960136488087349
iteration:  100  cost:  0.8509849107052945
iteration:  110  cost:  0.7750847670170908
iteration:  120  cost:  0.8205368143322392
iteration:  130  cost:  0.8482052776051655
iteration:  140  cost:  0.8277574343654966
Accuracy for U_TTN pca16 :0.9380614657210402
Epoch 1/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0944 - val_loss: 0.0289
Epoch 2/10
396/396 [==============================] - 1s 1ms/step - loss: 0.0276 - val_loss: 0.0228
Epoch 3/10
396/396 [==============================] - 1s 1ms/step 

#### 1.4 Binary Classification with 1, -1 labels with only Compact Encoding (3 rotation gates)

In [11]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_9', 'U_13', 'U_14', 'U_15', 'U_SO4']
U_num_params = [2, 10, 10, 4, 6, 6, 4, 6]
Encodings = ['pca24', 'autoencoder24']
classes = [0,1]
circuit = 'QCNN'

In [12]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = True)

0th step : 


Loss History for QCNN circuits, U_TTN pca24
iteration:  0  cost:  1.0414752772910194
iteration:  10  cost:  0.9900446631079973
iteration:  20  cost:  0.9748188462021049
iteration:  30  cost:  1.009169692711398
iteration:  40  cost:  0.9338012602156016
iteration:  50  cost:  1.002172471825047
iteration:  60  cost:  1.1654684796407944
iteration:  70  cost:  1.0093460558984744
iteration:  80  cost:  0.9860236106060475
iteration:  90  cost:  0.971159250392691
iteration:  100  cost:  0.9459460183732817
iteration:  110  cost:  0.92605720696656
iteration:  120  cost:  1.0199485848766627
iteration:  130  cost:  1.0000236129919007
iteration:  140  cost:  0.9992319724280523
Accuracy for U_TTN pca24 :0.5385342789598109
Epoch 1/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0867 - val_loss: 0.0254
Epoch 2/10
396/396 [==============================] - 0s 815us/step - loss: 0.0239 - val_loss: 0.0187
Epoch 3/10
396/396 [==============================] - 0s 784us/ste

In [13]:
results = [0.5385342789598109, 0.9016548463356974, 0.5399527186761229, 0.9011820330969267, 0.49598108747044917, 0.8543735224586289, 0.5366430260047281, 0.7924349881796691, 0.5810874704491725, 0.8761229314420804, 0.56548463356974, 0.858628841607565, 0.5432624113475177, 0.8591016548463357, 0.5366430260047281, 0.7858156028368795, 0.5366430260047281, 0.7319148936170212, 0.5366430260047281, 0.8775413711583925, 0.5366430260047281, 0.8860520094562647, 0.5631205673758866, 0.8756501182033097, 0.5385342789598109, 0.9134751773049645, 0.5394799054373522, 0.8879432624113475, 0.5366430260047281, 0.699290780141844, 0.5910165484633569, 0.8671394799054374, 0.5366430260047281, 0.631678486997636, 0.5366430260047281, 0.9106382978723404, 0.5366430260047281, 0.840661938534279, 0.4335697399527187, 0.8401891252955083, 0.5366430260047281, 0.8713947990543736, 0.5645390070921986, 0.7390070921985815, 0.5366430260047281, 0.8997635933806146, 0.5371158392434988, 0.9219858156028369, 0.5371158392434988, 0.7271867612293145, 0.5366430260047281, 0.9148936170212766, 0.5366430260047281, 0.8085106382978723, 0.5366430260047281, 0.6312056737588653, 0.5380614657210402, 0.9073286052009456, 0.5366430260047281, 0.9238770685579196, 0.5375886524822695, 0.8401891252955083, 0.577304964539007, 0.8548463356973995, 0.5366430260047281, 0.8278959810874704, 0.46004728132387707, 0.9120567375886525, 0.5366430260047281, 0.8761229314420804, 0.5366430260047281, 0.8170212765957446, 0.5366430260047281, 0.9328605200945627, 0.5366430260047281, 0.8713947990543736, 0.5375886524822695, 0.8945626477541371, 0.5366430260047281, 0.8288416075650118]

In [14]:
results_list = []
for i in range(16):
    #print(results[i], results[i+21], results[i+2*21], results[i+3*21], results[i+4*21])
    results_list.append([results[i], results[i+16], results[i+2*16], results[i+3*16], results[i+4*16]])

In [17]:
import statistics as st

In [18]:
print("Results for QNN with Compact-Compact Embedding (Ry-Rz-Ry) with 5 random initializations")
print("")
for i in range(16):
    if i < 2:
        it = "U_TTN"
    elif 1<i<4:
        it = 'U_5'
    elif 3<i<6:
        it = 'U_6'
    elif 5<i<8:
        it = 'U_9'
    elif 7<i<10:
        it = 'U_13'
    elif 9<i<12:
        it = 'U_14'
    elif 11<i<14:
        it = 'U_15'
    else:
        it = 'SO(4)'
    
    if i % 2 == 0:
        emb = "pca24"
    else:
        emb = "Autoencoding24"
        
    print(it + " " + emb + " Mean : " + str(st.mean(results_list[i])),  it + " " + emb + " Variance : " + str(st.variance(results_list[i])))

Results for QNN with Compact-Compact Embedding (Ry-Rz-Ry) with 5 random initializations

U_TTN pca24 Mean : 0.5371158392434988 U_TTN pca24 Variance : 6.706570762704779e-07
U_TTN Autoencoding24 Mean : 0.7640661938534279 U_TTN Autoencoding24 Variance : 0.01073129565358326
U_5 pca24 Mean : 0.5219858156028369 U_5 pca24 Variance : 0.0012009232712416659
U_5 Autoencoding24 Mean : 0.9032624113475177 U_5 Autoencoding24 Variance : 0.00023323217589099512
U_6 pca24 Mean : 0.5285106382978724 U_6 pca24 Variance : 0.0003306786490730961
U_6 Autoencoding24 Mean : 0.8531442080378251 U_6 Autoencoding24 Variance : 0.0009401047342800785
U_9 pca24 Mean : 0.521323877068558 U_9 pca24 Variance : 0.0025379452184944866
U_9 Autoencoding24 Mean : 0.7913002364066194 U_9 Autoencoding24 Variance : 0.008949538643819616
U_13 pca24 Mean : 0.546193853427896 U_13 pca24 Variance : 0.0003812014821521377
U_13 Autoencoding24 Mean : 0.9002364066193853 U_13 Autoencoding24 Variance : 0.000675798780521882
U_14 pca24 Mean : 0.5485

#### 1.5 Binary Classification with 1, -1 labels with U+P in the end

## 2. Results for Hierarchical Classifier circuit

#### 2.1 Binary Classification with 1, -1 labeling

In [19]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_13', 'U_14', 'U_15', 'U_SO4'] 
U_num_params = [2, 10, 10, 6, 6, 4, 6]
Encodings = ['resize256', 'pca8', 'autoencoder8']
classes = [0,1]
circuit = 'Hierarchical'

In [20]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = True)

0th step : 


Loss History for Hierarchical circuits, U_TTN resize256
iteration:  0  cost:  1.193148450071449
iteration:  10  cost:  0.8290276834444283
iteration:  20  cost:  0.9302363433562466
iteration:  30  cost:  0.986788512497954
iteration:  40  cost:  0.7107467410916322
iteration:  50  cost:  0.3353836269622367
iteration:  60  cost:  0.3645465184221521
iteration:  70  cost:  0.363218958892557
iteration:  80  cost:  0.28779898668935633
iteration:  90  cost:  0.34890035371694156
iteration:  100  cost:  0.37018830567516725
iteration:  110  cost:  0.22881955636152912
iteration:  120  cost:  0.3663425052375745
iteration:  130  cost:  0.34140589541114663
iteration:  140  cost:  0.32650150031342906
Accuracy for U_TTN resize256 :0.9725768321513002


Loss History for Hierarchical circuits, U_TTN pca8
iteration:  0  cost:  1.192942287160597
iteration:  10  cost:  0.9397208746087911
iteration:  20  cost:  0.9340626649607722
iteration:  30  cost:  0.6464723823486593
iteration:  40  cost:  0.

In [29]:
results = [0.9725768321513002, 0.9626477541371158, 0.7881796690307329, 0.9730496453900709, 0.9531914893617022, 0.8373522458628841, 0.9858156028368794, 0.9777777777777777, 0.9498817966903074, 0.9801418439716312, 0.6647754137115839, 0.9640661938534278, 0.9853427895981087, 0.6382978723404256, 0.9919621749408983, 0.9872340425531915, 0.9858156028368794, 0.9867612293144208, 0.9380614657210402, 0.983451536643026, 0.9839243498817967, 0.9706855791962175, 0.9692671394799054, 0.9749408983451536, 0.5947990543735224, 0.6392434988179669, 0.9631205673758865, 0.9796690307328605, 0.9541371158392435, 0.9952718676122931, 0.9791962174940898, 0.9569739952718677, 0.9229314420803783, 0.9475177304964539, 0.7423167848699763, 0.9498817966903074, 0.9895981087470449, 0.9843971631205674, 0.9654846335697399, 0.9886524822695035, 0.983451536643026, 0.9801418439716312, 0.975886524822695, 0.9801418439716312, 0.9167848699763593, 0.968321513002364, 0.6392434988179669, 0.8936170212765957, 0.9891252955082742, 0.9806146572104019, 0.9550827423167849, 0.9716312056737588, 0.6151300236406619, 0.9011820330969267, 0.9654846335697399, 0.9744680851063829, 0.9380614657210402, 0.9877068557919622, 0.9853427895981087, 0.991016548463357, 0.9385342789598109, 0.9872340425531915, 0.9872340425531915, 0.9739952718676123, 0.9456264775413712, 0.9546099290780142, 0.9735224586288416, 0.6042553191489362, 0.950354609929078, 0.9881796690307328, 0.9810874704491725, 0.9862884160756501, 0.957919621749409, 0.9810874704491725, 0.9617021276595744, 0.9858156028368794, 0.9588652482269504, 0.9749408983451536, 0.9886524822695035, 0.9862884160756501, 0.9678486997635933, 0.9891252955082742, 0.9810874704491725, 0.9697399527186761, 0.9777777777777777, 0.9527186761229315, 0.9602836879432625, 0.5366430260047281, 0.5583924349881797, 0.9947990543735225, 0.9886524822695035, 0.6548463356973995, 0.9900709219858156, 0.9796690307328605, 0.6033096926713948, 0.9513002364066194, 0.9881796690307328, 0.9815602836879432, 0.9513002364066194, 0.9891252955082742, 0.9839243498817967, 0.9773049645390071, 0.9881796690307328, 0.9867612293144208, 0.9739952718676123]

In [30]:
results_list = []
for i in range(21):
    #print(results[i], results[i+21], results[i+2*21], results[i+3*21], results[i+4*21])
    results_list.append([results[i], results[i+21], results[i+2*21], results[i+3*21], results[i+4*21]])

In [31]:
import statistics as st

In [32]:
results_list[0]

[0.9725768321513002,
 0.9706855791962175,
 0.975886524822695,
 0.9739952718676123,
 0.9777777777777777]

In [33]:
print("Results for a quantum hierarchical classifier with 5 random initializations")
print("")
for i in range(21):
    if i < 3:
        it = "U_TTN"
    elif 2<i<6:
        it = 'U_5'
    elif 5<i<9:
        it = 'U_6'
    elif 8<i<12:
        it = 'U_13'
    elif 11<i<15:
        it = 'U_14'
    elif 14<i<18:
        it = 'U_15'
    else:
        it = 'SO(4)'
    
    if i % 3 == 0:
        emb = "Resize"
    elif i % 3 == 1:
        emb = "PCA"
    else:
        emb = "Autoencoding"
        
    print(it + " " + emb + " Mean : " + str(st.mean(results_list[i])),  it + " " + emb + " Variance : " + str(st.variance(results_list[i])))

Results for a quantum hierarchical classifier with 5 random initializations

U_TTN Resize Mean : 0.9741843971631206 U_TTN Resize Variance : 7.66784590535913e-06
U_TTN PCA Mean : 0.962080378250591 U_TTN PCA Variance : 0.00018414007790799612
U_TTN Autoencoding Mean : 0.9189598108747045 U_TTN Autoencoding Variance : 0.0058051629417254875
U_5 Resize Mean : 0.8092671394799054 U_5 Resize Variance : 0.049855819906219775
U_5 PCA Mean : 0.6788652482269504 U_5 PCA Variance : 0.024618748218567143
U_5 Autoencoding Mean : 0.9278486997635934 U_5 Autoencoding Variance : 0.0038985966500679047
U_6 Resize Mean : 0.9862884160756501 U_6 Resize Variance : 1.531333657484258e-05
U_6 PCA Mean : 0.909692671394799 U_6 PCA Variance : 0.020421172643897863
U_6 Autoencoding Mean : 0.9753191489361702 U_6 Autoencoding Variance : 0.0004481554135998065
U_13 Resize Mean : 0.9737115839243499 U_13 Resize Variance : 9.015866628662704e-05
U_13 PCA Mean : 0.7642553191489362 U_13 PCA Variance : 0.03554869249814173
U_13 Autoen

#### 2.2 One Class Classification with 0, 1 labeling

In [13]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_13', 'U_14', 'U_15', 'U_SO4'] 
U_num_params = [2, 10, 10, 6, 6, 4, 6]
Encodings = ['resize256', 'pca8', 'autoencoder8']
classes = [0,1]
circuit = 'Hierarchical'

In [14]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = False)



Loss History for Hierarchical circuits, U_TTN resize256
iteration:  0  cost:  0.35525773742240896
iteration:  10  cost:  0.3328691558220538
iteration:  20  cost:  0.2140754008832847
iteration:  30  cost:  0.1015911831206335
iteration:  40  cost:  0.08523465681568673
iteration:  50  cost:  0.13504519589169214
iteration:  60  cost:  0.16959733491187542
iteration:  70  cost:  0.2056834277536673
iteration:  80  cost:  0.10870341952284272
iteration:  90  cost:  0.11761417128397657
iteration:  100  cost:  0.11893951068590494
iteration:  110  cost:  0.09069035817993487
iteration:  120  cost:  0.13341179349553453
iteration:  130  cost:  0.08403569285131886
iteration:  140  cost:  0.10402214866940401
Accuracy for U_TTN resize256 :0.9234042553191489


Loss History for Hierarchical circuits, U_TTN pca8
iteration:  0  cost:  0.4030474611440642
iteration:  10  cost:  0.2398302903707779
iteration:  20  cost:  0.19140073583434347
iteration:  30  cost:  0.2604377151220624
iteration:  40  cost:  0.17


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.





Loss History for Hierarchical circuits, U_TTN autoencoder8
iteration:  0  cost:  0.45104668114502133
iteration:  10  cost:  0.13006874763709894
iteration:  20  cost:  0.16286208703665153
iteration:  30  cost:  0.06777652117910357
iteration:  40  cost:  0.0459391601608008
iteration:  50  cost:  0.06394366327047958
iteration:  60  cost:  0.03721093113572069
iteration:  70  cost:  0.034127685274360886
iteration:  80  cost:  0.08091913406808816
iteration:  90  cost:  0.05133387049661154
iteration:  100  cost:  0.039230947766204144
iteration:  110  cost:  0.07935346013945002
iteration:  120  cost:  0.05189745380239168
iteration:  130  cost:  0.05803668770364166
iteration:  140  cost:  0.03936430964616579
Accuracy for U_TTN autoencoder8 :0.9687943262411347


Loss History for Hierarchical circuits, U_5 resize256
iteration:  0  cost:  0.997476893650433
iteration:  10  cost:  0.4954167402847351
iteration:  20  cost:  0.24272056847158224
iteration:  30  cost:  0.20005502694858276
iteration:  4


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.





Loss History for Hierarchical circuits, U_5 autoencoder8
iteration:  0  cost:  0.792473784379411
iteration:  10  cost:  0.14829486428336727
iteration:  20  cost:  0.09960659126649281
iteration:  30  cost:  0.07587366534578206
iteration:  40  cost:  0.13129787976785048
iteration:  50  cost:  0.170075390009835
iteration:  60  cost:  0.25372058195333985
iteration:  70  cost:  0.08029172844577716
iteration:  80  cost:  0.133766905830799
iteration:  90  cost:  0.11004888054169273
iteration:  100  cost:  0.1343455148924195
iteration:  110  cost:  0.08182072952662356
iteration:  120  cost:  0.0937487055403106
iteration:  130  cost:  0.1531691639080851
iteration:  140  cost:  0.15391712490631615
Accuracy for U_5 autoencoder8 :0.8770685579196218


Loss History for Hierarchical circuits, U_6 resize256
iteration:  0  cost:  0.28987397936802606
iteration:  10  cost:  0.24476557047361036
iteration:  20  cost:  0.221873622868841
iteration:  30  cost:  0.20406692830207832
iteration:  40  cost:  0.1

...



Loss History for Hierarchical circuits, U_14 autoencoder8
iteration:  0  cost:  0.3572586296063401
iteration:  10  cost:  0.19502081378433278
iteration:  20  cost:  0.20977013776562026
iteration:  30  cost:  0.21195527335313763
iteration:  40  cost:  0.11075903707560085
iteration:  50  cost:  0.13008799116386732
iteration:  60  cost:  0.11451330637085343
iteration:  70  cost:  0.10083753816851596
iteration:  80  cost:  0.1093915683024726
iteration:  90  cost:  0.10989801530048707
iteration:  100  cost:  0.09837303827764397
iteration:  110  cost:  0.1121862820059631
iteration:  120  cost:  0.0801396764777206
iteration:  130  cost:  0.08387410004447958
iteration:  140  cost:  0.07184851728393073
Accuracy for U_14 autoencoder8 :0.9706855791962175


Loss History for Hierarchical circuits, U_15 resize256
iteration:  0  cost:  0.5960445067441198
iteration:  10  cost:  0.1463659383104196
iteration:  20  cost:  0.13632372659031866
iteration:  30  cost:  0.10195681024820946
iteration:  40  co


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.





Loss History for Hierarchical circuits, U_15 autoencoder8
iteration:  0  cost:  0.1607200299821103
iteration:  10  cost:  0.10707737984189969
iteration:  20  cost:  0.04105760161061437
iteration:  30  cost:  0.04436563552708688
iteration:  40  cost:  0.029634263644470325
iteration:  50  cost:  0.030843134844376664
iteration:  60  cost:  0.030263129330024053
iteration:  70  cost:  0.02616273426498309
iteration:  80  cost:  0.014233070226215158
iteration:  90  cost:  0.025472571026347094
iteration:  100  cost:  0.05569407544679674
iteration:  110  cost:  0.019367488019393107
iteration:  120  cost:  0.02061968137896745
iteration:  130  cost:  0.010410629662882436
iteration:  140  cost:  0.03161753524011306
Accuracy for U_15 autoencoder8 :0.992434988179669


Loss History for Hierarchical circuits, U_SO4 resize256
iteration:  0  cost:  0.3553435337738612
iteration:  10  cost:  0.3320599358207073
iteration:  20  cost:  0.2607467730973488
iteration:  30  cost:  0.2345352265007858
iteration:


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.





Loss History for Hierarchical circuits, U_SO4 autoencoder8
iteration:  0  cost:  0.3510546701533224
iteration:  10  cost:  0.09761463420025823
iteration:  20  cost:  0.02397880549625883
iteration:  30  cost:  0.013904913720665116
iteration:  40  cost:  0.019979492217430412
iteration:  50  cost:  0.05444298196984325
iteration:  60  cost:  0.02591592070788897
iteration:  70  cost:  0.03392110830378884
iteration:  80  cost:  0.018468827989503687
iteration:  90  cost:  0.03045450012979467
iteration:  100  cost:  0.01612231112560783
iteration:  110  cost:  0.01679955287509967
iteration:  120  cost:  0.034568265798485064
iteration:  130  cost:  0.011824425365508871
iteration:  140  cost:  0.02467169435996436
Accuracy for U_SO4 autoencoder8 :0.9877068557919622


In [34]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_9', 'U_13', 'U_14', 'U_15', 'U_SO4']
U_num_params = [2, 10, 10, 4, 6, 6, 4, 6]
Encodings = ['pca24', 'autoencoder24']
classes = [0,1]
circuit = 'Hierarchical'

In [35]:
Benchmarking(Unitaries, U_num_params, Encodings, circuit, binary = True)

0th step : 


Loss History for Hierarchical circuits, U_TTN pca24
iteration:  0  cost:  0.9997996153381501
iteration:  10  cost:  0.9998064128078369
iteration:  20  cost:  0.9989525046880982
iteration:  30  cost:  0.9997559368239729
iteration:  40  cost:  0.9998278502347802
iteration:  50  cost:  1.0000842951290212
iteration:  60  cost:  0.9997679269654296
iteration:  70  cost:  0.9993545427635425
iteration:  80  cost:  0.998816429538643
iteration:  90  cost:  1.0002795163935976
iteration:  100  cost:  0.9991333403118513
iteration:  110  cost:  1.0006037428202683
iteration:  120  cost:  1.0000612875633228
iteration:  130  cost:  0.9992653894169533
iteration:  140  cost:  0.9994949833363068
Accuracy for U_TTN pca24 :0.574468085106383
Epoch 1/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0884 - val_loss: 0.0247
Epoch 2/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0235 - val_loss: 0.0189
Epoch 3/10
396/396 [==============================] - 1s 

/home/leeseok/.local/lib/python3.7/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


iteration:  0  cost:  1.004155630425517
iteration:  10  cost:  0.9801186313093001
iteration:  20  cost:  0.9884125088058524
iteration:  30  cost:  1.0212804057196612
iteration:  40  cost:  1.0185179702753564
iteration:  50  cost:  1.017015709009763
iteration:  60  cost:  1.0213386606990287
iteration:  70  cost:  0.9812997264625917
iteration:  80  cost:  0.9985003180322145
iteration:  90  cost:  0.9750897709318568
iteration:  100  cost:  1.0036768792964255
iteration:  110  cost:  1.0217111892627084
iteration:  120  cost:  1.0166359125604827
iteration:  130  cost:  1.015118002550057
iteration:  140  cost:  1.0105589937868844
Accuracy for U_9 pca24 :0.4988179669030733
Epoch 1/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0885 - val_loss: 0.0258
Epoch 2/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0244 - val_loss: 0.0188
Epoch 3/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0177 - val_loss: 0.0146
Epoch 4/10
396/396 [======

/home/leeseok/.local/lib/python3.7/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


iteration:  0  cost:  0.9973452681031474
iteration:  10  cost:  0.9731298761397481
iteration:  20  cost:  0.9696156124220093
iteration:  30  cost:  1.0025119370986753
iteration:  40  cost:  1.0173612455305416
iteration:  50  cost:  0.9868668567709828
iteration:  60  cost:  1.0068777164661527
iteration:  70  cost:  0.9934352750143159
iteration:  80  cost:  0.999264636898722
iteration:  90  cost:  1.0338102838160916
iteration:  100  cost:  0.9847054756924272
iteration:  110  cost:  0.9815042534389662
iteration:  120  cost:  1.0208361265833408
iteration:  130  cost:  1.0197417114191678
iteration:  140  cost:  0.9981657018377432
Accuracy for U_9 pca24 :0.4988179669030733
Epoch 1/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0855 - val_loss: 0.0260
Epoch 2/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0246 - val_loss: 0.0195
Epoch 3/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0184 - val_loss: 0.0148
Epoch 4/10
396/396 [====

...

iteration:  0  cost:  1.002967728972443
iteration:  10  cost:  1.0240474365667336
iteration:  20  cost:  0.9730328195750312
iteration:  30  cost:  0.9969858190687658
iteration:  40  cost:  1.0404855211821427
iteration:  50  cost:  1.0122706042522407
iteration:  60  cost:  1.004024611326056
iteration:  70  cost:  0.9942764401548103
iteration:  80  cost:  0.9983072910243636
iteration:  90  cost:  1.021248734511385
iteration:  100  cost:  1.0113942470219823
iteration:  110  cost:  1.028316364002478
iteration:  120  cost:  1.041763496522084
iteration:  130  cost:  1.000962120334127
iteration:  140  cost:  0.9938664971369717
Accuracy for U_9 pca24 :0.4988179669030733
Epoch 1/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0891 - val_loss: 0.0267
Epoch 2/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0247 - val_loss: 0.0191
Epoch 3/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0183 - val_loss: 0.0150
Epoch 4/10
396/396 [=========

/home/leeseok/.local/lib/python3.7/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


iteration:  0  cost:  1.0393626908627434
iteration:  10  cost:  1.0043454064270723
iteration:  20  cost:  0.9810154970928531
iteration:  30  cost:  1.0353666099942405
iteration:  40  cost:  1.022503651574295
iteration:  50  cost:  0.9714064596107633
iteration:  60  cost:  0.9884497977527457
iteration:  70  cost:  0.9898100098059968
iteration:  80  cost:  1.003980299243432
iteration:  90  cost:  0.9859406385189189
iteration:  100  cost:  0.9818148995278273
iteration:  110  cost:  1.013954670415737
iteration:  120  cost:  0.9883723436448122
iteration:  130  cost:  1.0305266384072977
iteration:  140  cost:  1.0325623232500727
Accuracy for U_9 pca24 :0.4988179669030733
Epoch 1/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0879 - val_loss: 0.0261
Epoch 2/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0246 - val_loss: 0.0197
Epoch 3/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0184 - val_loss: 0.0150
Epoch 4/10
396/396 [======

/home/leeseok/.local/lib/python3.7/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


iteration:  0  cost:  0.9804565338749075
iteration:  10  cost:  1.0121983173126123
iteration:  20  cost:  1.0212743275325167
iteration:  30  cost:  1.0413833096048994
iteration:  40  cost:  0.9823333564977923
iteration:  50  cost:  0.999428430878267
iteration:  60  cost:  0.9982094462270041
iteration:  70  cost:  1.0039761445716504
iteration:  80  cost:  1.0018317663314458
iteration:  90  cost:  1.0002301374350488
iteration:  100  cost:  0.9954367316815387
iteration:  110  cost:  0.9863874432654923
iteration:  120  cost:  1.0212201885544274
iteration:  130  cost:  1.0099469162289945
iteration:  140  cost:  0.9929014578071024
Accuracy for U_9 pca24 :0.4988179669030733
Epoch 1/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0877 - val_loss: 0.0263
Epoch 2/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0249 - val_loss: 0.0198
Epoch 3/10
396/396 [==============================] - 1s 2ms/step - loss: 0.0184 - val_loss: 0.0148
Epoch 4/10
396/396 [====

In [36]:
results = [0.574468085106383, 0.8250591016548463, 0.5366430260047281, 0.8539007092198582, 0.5451536643026005, 0.9163120567375886, 0.4988179669030733, 0.46335697399527187, 0.5470449172576832, 0.8832151300236407, 0.4912529550827423, 0.742789598108747, 0.4293144208037825, 0.902127659574468, 0.5366430260047281, 0.8893617021276595, 0.5328605200945626, 0.742789598108747, 0.5371158392434988, 0.8936170212765957, 0.5371158392434988, 0.957919621749409, 0.4988179669030733, 0.4628841607565012, 0.4302600472813239, 0.9432624113475178, 0.5721040189125296, 0.8723404255319149, 0.5394799054373522, 0.873758865248227, 0.5366430260047281, 0.8283687943262411, 0.5498817966903073, 0.8699763593380615, 0.5423167848699764, 0.908274231678487, 0.5366430260047281, 0.9087470449172577, 0.4988179669030733, 0.46335697399527187, 0.5380614657210402, 0.9054373522458629, 0.548936170212766, 0.8761229314420804, 0.5408983451536643, 0.9106382978723404, 0.5366430260047281, 0.8293144208037825, 0.49976359338061466, 0.7120567375886525, 0.5366430260047281, 0.8973995271867612, 0.5366430260047281, 0.8704491725768322, 0.4988179669030733, 0.46335697399527187, 0.5470449172576832, 0.8912529550827423, 0.5820330969267139, 0.9139479905437352, 0.5394799054373522, 0.915839243498818, 0.5366430260047281, 0.9706855791962175, 0.42080378250591016, 0.9130023640661938, 0.5366430260047281, 0.7692671394799054, 0.5366430260047281, 0.8666666666666667, 0.4988179669030733, 0.4595744680851064, 0.5394799054373522, 0.9361702127659575, 0.5375886524822695, 0.9186761229314421, 0.5408983451536643, 0.9513002364066194, 0.5366430260047281, 0.766903073286052]

In [37]:
results_list = []
for i in range(16):
    #print(results[i], results[i+21], results[i+2*21], results[i+3*21], results[i+4*21])
    results_list.append([results[i], results[i+16], results[i+2*16], results[i+3*16], results[i+4*16]])

In [38]:
print("Results for Hierarchical with Compact-Compact Embedding (Rz-Rz-Ry) with 5 random initializations")
print("")
for i in range(16):
    if i < 2:
        it = "U_TTN"
    elif 1<i<4:
        it = 'U_5'
    elif 3<i<6:
        it = 'U_6'
    elif 5<i<8:
        it = 'U_9'
    elif 7<i<10:
        it = 'U_13'
    elif 9<i<12:
        it = 'U_14'
    elif 11<i<14:
        it = 'U_15'
    else:
        it = 'SO(4)'
    
    if i % 2 == 0:
        emb = "pca24"
    else:
        emb = "Autoencoding24"
        
    print(it + " " + emb + " Mean : " + str(st.mean(results_list[i])),  it + " " + emb + " Variance : " + str(st.variance(results_list[i])))

Results for Hierarchical with Compact-Compact Embedding (Ry-Rz-Ry) with 5 random initializations

U_TTN pca24 Mean : 0.5155555555555555 U_TTN pca24 Variance : 0.0035439308329002026
U_TTN Autoencoding24 Mean : 0.8125768321513003 U_TTN Autoencoding24 Variance : 0.007127586919951489
U_5 pca24 Mean : 0.5378723404255319 U_5 pca24 Variance : 6.2147555734397624e-06
U_5 Autoencoding24 Mean : 0.8644917257683215 U_5 Autoencoding24 Variance : 0.0032570013804358133
U_6 pca24 Mean : 0.5384397163120568 U_6 pca24 Variance : 1.4128509073431402e-05
U_6 Autoencoding24 Mean : 0.9040189125295508 U_6 Autoencoding24 Variance : 0.0014002201990733756
U_9 pca24 Mean : 0.4988179669030733 U_9 pca24 Variance : 0.0
U_9 Autoencoding24 Mean : 0.46250591016548465 U_9 Autoencoding24 Variance : 2.727338776833277e-06
U_13 pca24 Mean : 0.5203782505910166 U_13 pca24 Variance : 0.0025552705262981406
U_13 Autoencoding24 Mean : 0.9118676122931442 U_13 Autoencoding24 Variance : 0.0007158817184469823
U_14 pca24 Mean : 0.546382